In [1]:
# Install required packages
#!pip install yfinance pandas numpy torch transformers pickle-mixin pyarrow
!pip install faiss-cpu
!pip install yfinance 

import yfinance as yf
import pandas as pd
import numpy as np
import torch
import json
import pickle
import re
from datetime import datetime
from itertools import groupby
from operator import itemgetter
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

import warnings
warnings.filterwarnings('ignore')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 103.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 13.3 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 133.5 MB/s eta 0:00:00a 0:00:01
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15548 sha256=2fe010a9532335bb73951b40657a62194948c33131be99de88686eb41016f8e8
  Stored in directory: /mnt/primary/launcher-cache/pip/wheels/e9/25/85/25d2e1cfc0ece64b930b16972f7e4cc3599c43b531f1eba06d
  Created wheel for peewee: filename=peewee-3.18.2-cp310-cp310-linux_x86_64.whl size=300916 sha256=e9af83325d3b6eb6fdbd6925d64b512e7fa1c034e93a0b57ec53900f4d530087
  Stored in directory: /mn

In [3]:
import yfinance as yf
import pandas as pd
import time

# List of 50 companies
companies = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "META", "TSLA", "NVDA", "JPM", "V", "WMT",
    "INTC", "AMD", "ADBE", "CRM", "ORCL", "IBM", "CSCO", "QCOM", "TSM", "NKE",
    "HD", "MCD", "SBUX", "DIS", "BKNG", "MA", "BAC", "WFC", "GS", "AXP",
    "JNJ", "PFE", "MRK", "ABT", "UNH", "LLY", "BA", "CAT", "HON", "GE",
    "UPS", "PG", "KO", "PEP", "XOM", "CVX", "COP", "T", "VZ", "NFLX"
]

def process_company_data(ticker, start_date, end_date):
    """Download and process data for a single company"""
    try:
        print(f"Processing {ticker}...")

        # Download data
        df = yf.download(ticker, start=start_date, end=end_date, auto_adjust=False)
        df.reset_index(inplace=True)

        # Clean column names
        df.columns = [f"{col[0]}_{col[1]}" if isinstance(col, tuple) and col[1] else col[0]
                     for col in df.columns]

        # Select and rename columns
        df = df[[
            'Date',
            f'Open_{ticker}',
            f'High_{ticker}',
            f'Low_{ticker}',
            f'Close_{ticker}',
            f'Volume_{ticker}',
            f'Adj Close_{ticker}'
        ]]

        df.rename(columns={
            f'Close_{ticker}': 'close',
            f'Open_{ticker}': 'open',
            f'High_{ticker}': 'high',
            f'Low_{ticker}': 'low',
            f'Volume_{ticker}': 'volume',
            f'Adj Close_{ticker}': 'adj_close',
            'Date': 'date'
        }, inplace=True)

        # Add ticker column
        df['ticker'] = ticker

        # Reorder columns
        df = df[['date', 'ticker', 'open', 'high', 'low', 'close', 'volume', 'adj_close']]

        # Save individual company file
        df.to_csv(f"{ticker}.csv", index=False)
        print(f"Saved {ticker}.csv")

        return df

    except Exception as e:
        print(f"Error processing {ticker}: {str(e)}")
        return None

def process_all_companies(companies, start_date, end_date):
    """Process all companies and combine into one file"""
    all_data = []

    for ticker in companies:
        df = process_company_data(ticker, start_date, end_date)
        if df is not None:
            all_data.append(df)
            time.sleep(0.2)  # Small delay to avoid rate limiting

    # Combine all data
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        combined_df.to_csv("all_companies.csv", index=False)
        print("Saved combined data to all_companies.csv")
        return combined_df
    else:
        print("No data was processed successfully")
        return None

# Main execution
if __name__ == "__main__":
    start_date = "2025-01-01"
    end_date = "2025-04-30"

    # Process all companies
    final_df = process_all_companies(companies, start_date, end_date)

    if final_df is not None:
        print("\nSample of combined data:")
        print(final_df.head())
        print("\nData processing complete!")

[*********************100%***********************]  1 of 1 completed

Processing AAPL...
Saved AAPL.csv



[*********************100%***********************]  1 of 1 completed

Processing MSFT...
Saved MSFT.csv



[*********************100%***********************]  1 of 1 completed

...

Saved T.csv
Processing VZ...


[*********************100%***********************]  1 of 1 completed


Saved VZ.csv
Processing NFLX...


[*********************100%***********************]  1 of 1 completed


Saved NFLX.csv
Saved combined data to all_companies.csv

Sample of combined data:
        date ticker        open        high         low       close    volume  \
0 2025-01-02   AAPL  248.929993  249.100006  241.820007  243.850006  55740700   
1 2025-01-03   AAPL  243.360001  244.179993  241.889999  243.360001  40244100   
2 2025-01-06   AAPL  244.309998  247.330002  243.199997  245.000000  45045600   
3 2025-01-07   AAPL  242.979996  245.550003  241.350006  242.210007  40856000   
4 2025-01-08   AAPL  241.919998  243.710007  240.050003  242.699997  37628900   

    adj_close  
0  242.987427  
1  242.499161  
2  244.133347  
3  241.353226  
4  241.841461  

Data processing complete!
